# EDA & Data Processing: Lyrics Dataset
**Project:** Moodio - Multimodal Emotion Recognition

## Tujuan
Setelah melakukan scraping lirik, kita perlu:
1. Membersihkan data (menghapus lagu yang tidak ada liriknya).
2. Menggabungkan data Lirik dengan Label Emosi (Valence/Arousal) dari dataset DEAM.
3. Melakukan analisis kata (Word Cloud) untuk melihat korelasi kata dengan emosi.
4. Menyimpan dataset final yang siap untuk training model NLP.

In [8]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# --- PERBAIKAN PATH (Wajib dijalankan paling awal) ---
# 1. Cek apakah kita ada di folder 'notebooks'? Jika ya, naik satu level ke root project.
if 'notebooks' in os.getcwd():
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    os.chdir(project_root)
    # Tambahkan root project ke sys.path agar Python bisa 'melihat' folder src
    if project_root not in sys.path:
        sys.path.append(project_root)
    print(f"✅ Berhasil pindah ke Root Directory: {os.getcwd()}")
else:
    print(f"ℹ️ Sudah berada di: {os.getcwd()}")

# 2. BARU lakukan import config dari folder src
try:
    import src.config as cfg
    print("✅ Berhasil import src.config")
except ImportError as e:
    print(f"❌ Gagal import config: {e}")
    print("   Pastikan file 'src/config.py' sudah dibuat!")

✅ Berhasil pindah ke Root Directory: c:\Users\andiz\OneDrive\Dokumen\Kuliah\Cawu4\[CAWU 4] GROUP 3\projectRoodio\machineLearning
✅ Berhasil import src.config


In [10]:
# 1. Load Hasil Scraping Lirik
lyrics_path = os.path.join(cfg.BASE_DIR, "data", "processed", "lyrics", "lyrics_dataset_raw.csv")
df_lyrics = pd.read_csv(lyrics_path)

print(f"Total baris scraping: {len(df_lyrics)}")
print(f"Lagu dengan lirik ditemukan: {df_lyrics['lyrics'].notna().sum()}")

# 2. Load Label Emosi (Valence & Arousal)
# Karena DEAM datanya per detik (dynamic), kita perlu merata-ratanya agar jadi SATU nilai per lagu
print("\nMemproses Label Emosi (Averaging)...")

def load_and_average_labels(filename):
    path = os.path.join(cfg.ANNOTATIONS_DIR, filename)
    df = pd.read_csv(path)
    # Format DEAM: sample_15000ms, sample_15500ms...
    # Kita ambil rata-rata semua kolom waktu untuk setiap lagu
    # Filter hanya kolom yang mengandung 'sample_'
    time_cols = [c for c in df.columns if 'sample_' in c]
    
    # Hitung rata-rata per baris (per lagu)
    df['mean_value'] = df[time_cols].mean(axis=1)
    return df[['song_id', 'mean_value']]

df_valence = load_and_average_labels("valence.csv")
df_arousal = load_and_average_labels("arousal.csv")

# Gabungkan Valence & Arousal
df_labels = pd.merge(df_valence, df_arousal, on='song_id', suffixes=('_valence', '_arousal'))
df_labels.rename(columns={'mean_value_valence': 'Valence', 'mean_value_arousal': 'Arousal'}, inplace=True)

print(f"Total Label Emosi: {len(df_labels)}")
display(df_labels.head())

Total baris scraping: 1129
Lagu dengan lirik ditemukan: 175

Memproses Label Emosi (Averaging)...
Total Label Emosi: 1802


,song_id,Valence,Arousal
0,2,-0.215511,-0.197517
1,3,-0.265855,-0.193187
2,4,0.155210,0.243072
3,5,0.140160,-0.236207
4,7,0.332455,0.376292
